### 03. 다중 선형 회귀(Multivariable Linear regression)
앞서 배운 x가 1개인 선형 회귀를 단순 선형 회귀라고 한다. 이제는 다수의 x로부터 y를 예측하는 다중 선형 회귀에 대해서 공부

#### 1. 데이터에 대한 이해
- 훈련 데이터 : 독립 변수 x의 개수가 3개로 3개의 퀴즈 점수로부터 최종 점수를 예측하는 모델 만들기
- 식 : H(x) = w1x1 + w2x2 + w3x3 + b

------------------------------------------------------------------------------------------------------------

#### 2. 파이토치로 구현하기

In [3]:
# 필요한 도구들을 임포터하고 랜덤 시드 고정
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
torch.manual_seed(1)

In [9]:
# 훈련 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [10]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad = True)
w2 = torch.zeros(1, requires_grad = True)
w3 = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [13]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1) : 
    
    # H(x) 계산
    hypothesis = x1_train*w1 + x2_train*w2 + x3_train*w3 + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) **2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 : 
        print('Epoch {:4d}/{} w1 : {:.3f} w2 : {:.3f} w3 : {:.3f} b : {:.3f} Cost : {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))
    

Epoch    0/1000 w1 : 0.718 w2 : 0.612 w3 : 0.680 b : 0.009 Cost : 1.078544
Epoch  100/1000 w1 : 0.722 w2 : 0.608 w3 : 0.680 b : 0.009 Cost : 1.037795
Epoch  200/1000 w1 : 0.727 w2 : 0.603 w3 : 0.681 b : 0.010 Cost : 0.999138
Epoch  300/1000 w1 : 0.731 w2 : 0.599 w3 : 0.681 b : 0.010 Cost : 0.962497
Epoch  400/1000 w1 : 0.735 w2 : 0.595 w3 : 0.681 b : 0.010 Cost : 0.927747
Epoch  500/1000 w1 : 0.739 w2 : 0.590 w3 : 0.681 b : 0.010 Cost : 0.894806
Epoch  600/1000 w1 : 0.743 w2 : 0.586 w3 : 0.682 b : 0.010 Cost : 0.863567
Epoch  700/1000 w1 : 0.746 w2 : 0.582 w3 : 0.682 b : 0.010 Cost : 0.833937
Epoch  800/1000 w1 : 0.750 w2 : 0.579 w3 : 0.682 b : 0.010 Cost : 0.805833
Epoch  900/1000 w1 : 0.754 w2 : 0.575 w3 : 0.682 b : 0.010 Cost : 0.779167
Epoch 1000/1000 w1 : 0.757 w2 : 0.571 w3 : 0.682 b : 0.011 Cost : 0.753894


------------------------------------------------------------------------------------------------------------

#### 3. 벡터와 행렬 연산으로 바꾸기
- 위의 코드를 개선할 수 있는 부분 !
    - x의 개수가 3개였으니까, x1_train, x2_train, x3_train, w1, w2, w3을 일일히 선언했다.
    - 하지만 x의 개수가 1000개라면 ? -> 행렬 곱셈 연산(또는 벡터의 내적)을 사용
- 행렬 곱셈 과정에서 이루어지는 벡터 연산을 백터의 내적이라고 한다.
    - 가설을 벡터와 행렬 연산으로 표현할 수 있다 !

##### 1. 벡터 연산으로 이해하기
H(X) = w1x1 + w2x2 + w3x3
- 위 식은 아래와 같이 두 벡터의 내적으로 표현할 수 있다.
- 두 벡터를 각각 X와 W로 표현한다면, 가설은 : H(X) = XW
- x의 개수가 3개였지만 이제는 X와 W라는 두 개의 변수로 표현된다.

##### 2. 행렬 연산으로 이해하기
- 전체 훈련 데이터의 개수를 셀 수 있는 1개의 단위를 샘플(sample)이라고 한다. (현재 샘플의 수는 총 5개)
- 각 샘플에서 y를 결정하게 하는 각각의 독립 변수 x를 특성(feature)이라고 한다. (현재 특성은 3개)
1. 이는 독립 변수 x들의 수가 15개임을 의미하고 독립변수 x들을 (샘플의 수 x 특성의 수)의 크기를 가지는 하나의 행렬로 표현 -> X
2. 여기에 가중치 w1, w2, w3을 원소로 하는 벡터를 W라고 하고 이를 곱한다.
3. 샘플 수만큼 차원을 가지는 편향 벡터 B를 만들어 더한다.
- 위 식의 결과는 H(X) = XW + B로 전체 훈련 데이터의 가설 연산을 3개의 변수로만 표현

------------------------------------------------------------------------------------------------------------

#### 4. 행렬 연산을 고려하여 파이토치로 구현하기

In [16]:
# 훈련 데이터 또한 행렬로 선언
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80], 
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [17]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [21]:
# 가중치와 편향 선언
W = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [22]:
# 가설을 행렬곱으로 간단히 정의
hypothesis = x_train.matmul(W) + b

In [27]:
# 비용함수와 옵티마이저를 정의하고 정해진 에포크만큼 훈련 진행
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80], 
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1) :
    
    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해진다.
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) **2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis : {} Cost : {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis : tensor([0., 0., 0., 0., 0.]) Cost : 29661.800781
Epoch    1/20 hypothesis : tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost : 9537.694336
Epoch    2/20 hypothesis : tensor([104.5421, 125.6208, 119.2478, 134.7861,  95.8280]) Cost : 3069.590820
Epoch    3/20 hypothesis : tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost : 990.670288
Epoch    4/20 hypothesis : tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost : 322.481964
Epoch    5/20 hypothesis : tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost : 107.717064
Epoch    6/20 hypothesis : tensor([148.9423, 178.9731, 169.8976, 192.0301, 136.5279]) Cost : 38.687401
Epoch    7/20 hypothesis : tensor([151.1574, 181.6347, 172.4254, 194.8856, 138.5585]) Cost : 16.499046
Epoch    8/20 hypothesis : tensor([152.4131, 183.1435, 173.8590, 196.5042, 139.7097]) Cost : 9.365656
Epoch    9/20 hypothesis : tensor([153.1250, 183.9988, 174.6723, 197.4216, 140.3625]) Cost : 7.0711